In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report

from torch.utils.data import Subset


### Carregamento e Pré-Processamento do Dataset

In [ ]:
from utils import data_utils


image_size = (224, 224)
train_data, test_data = data_utils.load_and_preprocess_data(resize_to=image_size, keras_format=True)

### Divisão Holdout

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(train_data.targets)), train_data.targets))


In [ ]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [ ]:
print(f"Treino:    {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste:     {len(test_data)} imagens")

In [ ]:
def get_X_y(dataset):
    X = []
    y = []
    for img, label in dataset:
        X.append(img.numpy())  
        y.append(label)
    X = np.stack(X)
    y = np.array(y)
    return X, y


In [ ]:
X_train, y_train = get_X_y(train_split)
X_val, y_val = get_X_y(val_split)
X_test, y_test = get_X_y(test_data)

## Treino

### Modelo

**EfficientNetB0**

lorem ypslom


In [ ]:
device = "GPU" if len(tf.config.list_physical_devices('GPU')) > 0 else "CPU"
print(f"Usando: {device}")

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') 
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    verbose=1
)

In [ ]:
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))

roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC (ROC): {roc_auc:.4f}")

plt.figure(figsize=(8, 4))
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title("Acurácia")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend()
plt.grid()
plt.show()